In [1]:
%pip install psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install pandas sqlalchemy psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import numpy as np
import os
import glob 
from sqlalchemy import create_engine

# SAFE DATA LOADING 
raw_data_path = '/Users/annycarmen/sat-results.csv'
df = None

if os.path.exists(raw_data_path):
    df = pd.read_csv(raw_data_path)
    print("✅ Success! File loaded from specific path.")
else:
    search_pattern = '/Users/annycarmen/**/sat-results.csv'
    files = glob.glob(search_pattern, recursive=True)
    if files:
        df = pd.read_csv(files[0])
        print(f"✅ Success! Found via search at: {files[0]}")
    else:
        raise FileNotFoundError("❌ CRITICAL: The file 'sat-results.csv' was not found anywhere.")

# DROP IRRELEVANT COLUMNS 
# Names from your dataset structure documentation
cols_to_drop = ['SAT Critical Readng Avg. Score', 'internal_school_id', 'contact_extension']
df_clean = df.drop(columns=cols_to_drop, errors='ignore').copy()

# STANDARDIZE HEADERS
df_clean.columns = [
    'dbn', 'school_name', 'num_test_takers', 
    'reading_avg', 'math_avg', 'writing_avg', 
    'pct_tested', 'academic_rating'
]

# CLEAN PERCENTAGES, 's' and 'N/A'
# Converting "85%" string to float 0.85
df_clean['pct_tested'] = df_clean['pct_tested'].astype(str).str.replace('%', '').replace(['N/A', 'nan', 's'], np.nan)
df_clean['pct_tested'] = pd.to_numeric(df_clean['pct_tested'], errors='coerce') / 100

# VALIDATE SAT SCORES (200-800) 
# Scores must be within 200-800 per documentation
score_cols = ['reading_avg', 'math_avg', 'writing_avg']
for col in score_cols:
    df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
    df_clean.loc[(df_clean[col] < 200) | (df_clean[col] > 800), col] = np.nan

# DATA INTEGRITY (DEDUPLICATION) 
before_dedup = len(df_clean)
df_clean = df_clean.drop_duplicates(subset=['dbn'])
print(f"Data Integrity: Removed {before_dedup - len(df_clean)} duplicate records.")

# SAVE CLEANED CSV 
df_clean.to_csv('cleaned_sat_results.csv', index=False)
print(f"Step 1: cleaned_sat_results.csv generated successfully.")

# Database Configuration (Neon Credentials)
DB_CONFIG = {
    'host': 'ep-falling-glitter-a5m0j5gk-pooler.us-east-2.aws.neon.tech',
    'port': 5432,
    'database': 'neondb',
    'user': 'neondb_owner',
    'password': 'a9Am7Yy5r9_T7h4OF2GN'
}

def create_connection():
    try:
        conn_url = f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}?sslmode=require"
        engine = create_engine(conn_url)
        return engine
    except Exception as e:
        print(f"❌ Connection failed: {e}")
        return None

# Initialize the engine
engine = create_connection()   

if engine is not None and 'df_clean' in locals():
    try:
        table_name = 'annyhllosa_sat_scores'
        df_clean.to_sql(table_name, engine, if_exists='replace', index=False)
        print(f"Success! Table '{table_name}' updated in Neon.)")
    except Exception as e:
        print(f"Error inserting data: {e}")  
 

✅ Success! File loaded from specific path.
Data Integrity: Removed 15 duplicate records.
Step 1: cleaned_sat_results.csv generated successfully.
Success! Table 'annyhllosa_sat_scores' updated in Neon.)
